In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Embedding, Dropout

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Embedding, Dropout

In [3]:
import glob
filelist = glob.glob('data/Tagore/data/*.txt')

text_data = []
for file in filelist:
    with open(file, 'rb') as file:
        file_content = file.read().decode('utf-8')
        text_data.append(file_content)
        
len(text_data)

20

In [4]:
text_data[1]

'The Project Gutenberg EBook of Creative Unity, by Rabindranath Tagore\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Creative Unity\r\n\r\nAuthor: Rabindranath Tagore\r\n\r\nRelease Date: October 21, 2007 [EBook #23136]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: ASCII\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK CREATIVE UNITY ***\r\n\r\n\r\n\r\n\r\nProduced by Michael Ciesielski, Irma Špehar and the Online\r\nDistributed Proofreading Team at https://www.pgdp.net\r\n\r\n\r\n\r\n\r\n\r\n                            CREATIVE UNITY\r\n\r\n                                  BY\r\n\r\n                          RABINDRANATH TAGORE\r\n\r\n                      MACMILLAN AND CO., LIMITED\r\n                      ST. MARTIN\'S STREET, LONDON\r\n  

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word
words_count = len(word_idx)

In [7]:
words_count

29566

In [8]:
sequences = tokenizer.texts_to_sequences(text_data)
features = []
labels = []
look_back = 20
for seq in sequences:
    for i in range(len(seq)-look_back-1):
        values = seq[i: i+(look_back)]
        features.append(values)
        label = seq[i+look_back]
        labels.append(label)


In [9]:
features[1], labels[1]

([57,
  43,
  256,
  3,
  2068,
  37,
  544,
  729,
  1,
  17,
  256,
  9,
  16,
  2,
  169,
  3,
  752,
  1218,
  32,
  44],
 650)

In [10]:
from sklearn.utils import shuffle

In [11]:
# Shuffling and getting only limited features and labels
final_features, labels = shuffle(features, labels, random_state=1)
final_features = np.array(final_features[:10000])
labels = np.array(labels[:10000])


In [12]:
print(features[0], labels[0])

[2, 57, 43, 256, 3, 2068, 37, 544, 729, 1, 17, 256, 9, 16, 2, 169, 3, 752, 1218, 32] 5993


In [13]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(final_features, labels, 
                                                                            test_size=0.2, random_state=1)

In [14]:
# Do a onehot encoding

train_labels_onehot = np.zeros((len(train_labels), words_count))
test_labels_onehot = np.zeros((len(test_labels), words_count))


In [15]:
for example_index, word_index in enumerate(train_labels):
    train_labels_onehot[example_index, word_index] = 1
    
for example_index, word_index in enumerate(test_labels):
    test_labels_onehot[example_index, word_index] = 1

In [16]:
[idx_word[word] for word in train_features[1]], [idx_word[train_labels[1]]]

(['for',
  'my',
  "life's",
  'story',
  'i',
  'then\r',
  'discovered',
  'as',
  'i',
  'opened',
  'the',
  'door',
  'that',
  "life's",
  'memories',
  'are',
  'not',
  "life's\r",
  'history',
  'but'],
 ['the'])

## Build the model

In [2]:
np.__version__

'1.19.5'

In [ ]:
#!pip install --user numpy==1.19.5

In [17]:
model = Sequential()
model.add(Embedding(input_dim=words_count, output_dim=50, trainable=True, input_length=20))
model.add(LSTM(64, dropout=0.1, recurrent_dropout=0.1, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(words_count, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(train_features, train_labels_onehot, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
125/125 [==============================] - 11s 68ms/step - loss: 8.3610 - accuracy: 0.0535
Epoch 2/10
125/125 [==============================] - 9s 69ms/step - loss: 6.9860 - accuracy: 0.0620
Epoch 3/10
125/125 [==============================] - 9s 69ms/step - loss: 6.7206 - accuracy: 0.0616
Epoch 4/10
125/125 [==============================] - 9s 69ms/step - loss: 6.5780 - accuracy: 0.0622
Epoch 5/10
125/125 [==============================] - 9s 69ms/step - loss: 6.4452 - accuracy: 0.0620
Epoch 6/10
125/125 [==============================] - 9s 69ms/step - loss: 6.3446 - accuracy: 0.0631
Epoch 7/10
125/125 [==============================] - 8s 68ms/step - loss: 6.2189 - accuracy: 0.0640
Epoch 8/10
125/125 [==============================] - 9s 68ms/step - loss: 6.1098 - accuracy: 0.0636
Epoch 9/10
125/125 [==============================] - 9s 69ms/step - loss: 6.0018 - accuracy: 0.0664
Epoch 10/10
125/125 [==============================] - 9s 68ms/step - loss: 5.9288 - accur